In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sea
import tensorflow as tf
import random
import gc

import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, ReLU, BatchNormalization
from tensorflow.keras.initializers import HeNormal, Ones, Zeros
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
SEED = 3126  
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Initialize processes

In [3]:
WIDTH, HEIGHT = 224, 224
BATCH_SIZE = 32
LEARNING_RATE = .01
EPOCHS = 25
DIR = "/kaggle/input/upsampled-astrovision-data"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    DIR,
    target_size=(WIDTH, HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    seed=SEED  
)

validation_generator = train_datagen.flow_from_directory(
    DIR,
    target_size=(WIDTH, HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    seed=SEED 
)

Found 11225 images belonging to 4 classes.
Found 2804 images belonging to 4 classes.


# Initialize Model

In [4]:
def make_dense_layer(input_size, dropout_rate=0.0, input_shape=None):
    layers = []
    if input_shape:
        layers.append(Dense(input_size,
                            use_bias=False, 
                            input_shape=input_shape,
                            kernel_initializer=HeNormal()
                            ))
    else:
        layers.append(Dense(input_size, use_bias=False, kernel_initializer=HeNormal()))
    layers.extend([BatchNormalization(gamma_initializer=Ones(), beta_initializer=Zeros()),
                  ReLU()])
    if dropout_rate > 0:
        layers.append(Dropout(dropout_rate))
    return Sequential(layers)

In [5]:
def build_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False,
                             input_shape=(WIDTH, HEIGHT, 3))
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    fc_layers = Sequential([
        make_dense_layer(1024, input_shape=(x.shape[-1],)),
        make_dense_layer(512),
        make_dense_layer(256),
        make_dense_layer(32),
        make_dense_layer(8)
    ])
    x = fc_layers(x)
    predictions = Dense(train_generator.num_classes,
                        activation='softmax',
                        kernel_initializer=HeNormal())(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])   
    return model

# Trains Model

In [6]:
class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        K.clear_session()  
        gc.collect() 

In [7]:
model = build_model()

best_weights_path = "/kaggle/working/best_weights.weights.h5"

checkpoint_callback = ModelCheckpoint(
    filepath=best_weights_path,  # Path to save the model
    monitor='val_accuracy',    # Metric to monitor (can also use 'val_loss')
    save_best_only=True,       # Save only the best model
    save_weights_only=True,    # Save only the weights (not the full model)
    mode='max',                # 'max' because higher accuracy is better
    verbose=1                  # Print messages when saving the model
)

early_stopping_callback = EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=10,              # Number of epochs with no improvement before stopping
    mode='min',              # 'max' because higher accuracy is better
    min_delta=.001,
    restore_best_weights=True,  # Restores the best weights when stopping
    verbose=1
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[checkpoint_callback, early_stopping_callback, ClearMemory()]
)

initial_test_loss, initial_test_acc = model.evaluate(validation_generator, 
                                                     steps=validation_generator.samples // BATCH_SIZE)

print(f"Test Accuracy Before Fine-tuning: {initial_test_acc*100:.2f}%")

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


350/351 ━━━━━━━━━━━━━━━━━━━━ 0s 503ms/step - accuracy: 0.6844 - loss: 0.7373
Epoch 1: val_accuracy improved from -inf to 0.83024, saving model to /kaggle/working/best_weights.weights.h5
351/351 ━━━━━━━━━━━━━━━━━━━━ 245s 648ms/step - accuracy: 0.6847 - loss: 0.7365 - val_accuracy: 0.8302 - val_loss: 0.4528
Epoch 2/25
350/351 ━━━━━━━━━━━━━━━━━━━━ 0s 399ms/step - accuracy: 0.7969 - loss: 0.4706
Epoch 2: val_accuracy did not improve from 0.83024
351/351 ━━━━━━━━━━━━━━━━━━━━ 178s 497ms/step - accuracy: 0.7969 - loss: 0.4706 - val_accuracy: 0.7964 - val_loss: 0.4389
Epoch 3/25
350/351 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step - accuracy: 0.7995 - loss: 0.4518
Epoch 3: val_accuracy did not improve from 0.83024
351/351 ━━━━━━━━━━━━━━━━━━━━ 169s 473ms/step - accuracy: 0.7995 - loss: 0.4518 - val_accuracy: 0.8042 - val_loss: 0.4447
Epoch 4/25
350/351 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step - accuracy: 0.8064 - loss: 0.4309
Epoch 4: val_accuracy did not improve from 0.83024
351/351 ━━━━━━━━━━━━━━━━━━━━ 169s 